# Spiral of Numbers
Thanks to [@Bartłomiej Burek](https://www.facebook.com/bartlomiej.burek?fref=ufi) - who recognized the pattern in this post http://tinyurl.com/y8sdk6l3, the idea of this exercise was conceived.

## The goal of the exercise

Create a function that receives an integer number and builds a spiral of numbers - up to the provided number, starting to the right and turning counterclockwise - like that

```
37 36 35 34 33 32 31
38 17 16 15 14 13 30
39 18  5  4  3 12 29
40 19  6  1  2 11 28
41 20  7  8  9 10 27
42 21 22 23 24 25 26
43 44 45 46 47 48 49
```
The function output may be:
* list of lists 
```
[[17, 16, 15, 14, 13],
[18, 5, 4, 3, 12],
[19, 6, 1, 2, 11, 28],
[20, 7, 8, 9, 10, 27],
[21, 22, 23, 24, 25, 26]]
```
* "flattened" spiral - (5, 4, 3, 6, 1, 2, 7, 8)

* simple string output (as above)

* fancy string output 
```
5 <- 4 <- 3
|         ^
V         |
6    1 -> 2    11
|               ^
V               |
7 -> 8 -> 9 -> 10
```

The test harness for function testing is available at https://repl.it/@volvano63/NumericSpiralTestHarness and below

Good luck! I hope you'll enjoy the challenge

# Collected Solutions

You may peek - but you will ruin the challenge to yourself!

In [1]:
def Mark_Geyzer_spiral(number, as_matrix=True):
    spiral_body=[]
    def _build_spiral_helper(value=1, spiral_step=1, direction='right'):
        if value > number:
            return direction in 'upright'
        advance_to = min(number + 1, value + spiral_step)
        leap_range = range(value, advance_to)
        if direction == 'right':
            spiral_body.append(list(leap_range))
            direction = 'up'
        elif direction == 'up':
            for row, insert_value in zip(reversed(spiral_body), leap_range):
                row.append(insert_value)
            direction = 'left'
            spiral_step += 1
        elif direction == 'left':
            spiral_body.insert(0, list(reversed(leap_range)))
            direction = 'down'
        elif direction == 'down':
            for row, insert_value in zip(spiral_body, leap_range):
                row.insert(0, insert_value)                  
            direction = 'right'
            spiral_step += 1
        return _build_spiral_helper(advance_to, spiral_step, direction)
    
    align_left = _build_spiral_helper()
    if as_matrix:
        return spiral_body
    cell_formatter = '{{:>{}}}'.format(len(str(number)))
    spiral_lines = [' '.join(cell_formatter.format(cell) for cell in row)
                    for row in spiral_body]
    if align_left:
        return '\n'.join(spiral_lines)
    max_line_len = max(len(l) for l in spiral_lines)
    return '\n'.join('{:>{length}}'.format(l, length=max_line_len) for l in spiral_lines)


In [2]:
import math
def Bartlomiej_Burek_spiral(numbers, empty_rows=False, DEBUG=False):

    if DEBUG:
        print('[DEBUG] numbers:', numbers)

    # size of square matrix (to keep all values)
    size = math.ceil(math.sqrt(numbers))
    if DEBUG:
        print('[DEBUG] matrix size:', size)

    data = [[''] * size for _ in range(size)]
    #print('data:', data)

    # start positon for value `1` 
    center = math.floor((size-1) / 2)
    x = center
    y = (size - 1) - center # flip up-down

    if DEBUG:
        print('[DEBUG] start:', x, y, '(x,y)') 

    # (move_y,move_x) for "right", "up", "left", "down"
    moves = [(0,1), (-1,0), (0,-1), (1,0)]
    
    # every move has to be repeated: 1,1,2,2,3,3,4,4,5,5,etc.
    # so it will be [1,1], later [1,2], later [2,2], later [2,3], etc.
    repeats = [1, 1]

    # put `1` in matrix in starting position
    value = 1
    data[y][x] = value

    # put other values
    while value < numbers:
        # get direction from beginning of list
        move_y, move_x = moves.pop(0)
        # get how many times to move in this direction - get it from beginning of list
        repeat = repeats.pop(0)
        
        for _ in range(repeat):
            # get next value
            value += 1
            # move to new place
            y += move_y 
            x += move_x
            # put value in matrix
            data[y][x] = value
            
            # for some values you have to make less repeates at the end
            if value >= numbers:
                break
                
        # put repeat+1 at the end of list
        repeats.append(repeat+1)        
        # put direction at the end of list
        moves.append((move_y, move_x))    

    # added later: it removes empty rows. 
    # Code passes test without filter too.
    # Test needs `list()` to get correct data from `filter()`
    if not empty_rows:
        data = list(filter(any, data))
    
    return data


In [3]:
def Arthur_Zopellaro_spiral(list_size):
    from math import floor, sqrt

    values = []
    max_size = 2    # first line limited to 2 entries
    position = 0    # 0 bot, 1 right, 2 top, 3 left
    side_count = 1  # helper to append side values correctly
    for i in range(1, list_size + 1):
        current_size = len(values)
        while current_size == max_size:
            position = (position + 1) % 4
            index = floor(sqrt(i - 2))
            if position in (1, 3):
                side_count = 1
                max_size = index * (index + 1)
                line_length = (2 * floor(sqrt(current_size / 4))) + 1
            else:
                max_size = (index ** 2) + (2 * index) + 2
 
        if position == 0:
            values.append(i)
        elif position == 2:
            values.insert(0, i)
        elif position == 1:
            side_position = (line_length + 1) * side_count
            side_count += 1
            values.insert(-side_position, i)
        else:
            side_position = line_length * side_count
            side_count += 1
            values.insert(side_position, i)
    return values


In [4]:
def ALex_Ho_Spiral(number):
    dir_dict = {0: [0, -1], 1: [-1, 0], 2: [0, 1], 3: [1, 0]}

    def get_dimen(num):
        if num < 10:
            return 3
        i = 0
        j = 0
        while j < num:
            j = (3 + i ) ** 2
            i += 2
        return i+1

    def get_center(dimen):
        return int((dimen - 1) / 2)

    def turn_left(direction):
        dir_ = (direction + 1) % 4
        return dir_

    def move(direction, x, y, matrix, value):
        dx, dy = dir_dict[direction]
        x += dx
        y += dy
        value += 1
        matrix[x][y] = value
        return matrix, x, y, value

    def is_left_empty(direction, x, y, matrix):
        dir_ = turn_left(direction)
        dx, dy = dir_dict[dir_]
        x += dx
        y += dy
        return isinstance(matrix[x][y], str)

    def run(num):
        digits = len(str(num))
        dimen = get_dimen(num)
        x = y = get_center(dimen)
        value = 1
        direction = 3
        matrix = [["*" * digits] * dimen for _ in range(dimen)]
        matrix[x][y] = 1
        matrix, x, y, value = move(direction, x, y, matrix, value)
        while value < num:
            if is_left_empty(direction, x, y, matrix):
                direction = turn_left(direction)
                matrix, x, y, value = move(direction, x, y, matrix, value)
            else:
                matrix, x, y, value = move(direction, x, y, matrix, value)
        return matrix
    return list(zip(*run(number)))


In [5]:
import math
def Julie_Deeley_spiral(spiral_size):
    rows = round(math.sqrt(spiral_size))  # the number of 'rows' or sublists required
    sub_elements = int(math.ceil(spiral_size/rows))  # the number of elements in each sub-list
    alist = [['*'for _ in range(sub_elements)] for _ in range(rows)]  # creates placeholder list
    number = 1  # the number to begin the spiral with
    move_number = 0  # keeps track of the  number of moves for each direction
    sub = int(math.floor(rows/2))  # find the correct sublist to put the 1 in.
    elem = int(math.ceil(sub_elements/2))-1  # find the correct position within the sublist to put the 1
    alist[sub][elem] = 1  # place the 1

    while True:
        for direction in range(1, 5):  # 1= right, 2= up, 3= left, 4=down
            if direction in (1, 3):
                move_number += 1
            for number in range(number + 1, number + move_number + 1):
                if number > spiral_size:
                    return alist
                if direction == 1:
                    elem += 1
                elif direction == 2:
                    sub -= 1
                elif direction == 3:
                    elem -= 1
                elif direction == 4:
                    sub += 1
                alist[sub][elem] = number


In [6]:
def Julie_Deeley_spiral_on_the_fly(spiral_size):
    """ This version doesn't create lists in advance"""
    def move_range():
        return range(number + 1, min(number + move_number + 1, spiral_size + 1))

    number = 1          # start number placed in spiral
    alist = [[number]]          # start a list
    move_number = 1     # a move counter, increases for every pair of moves
    sub = 0             # the sublist index number in list alist
    
    while number < spiral_size:       
        for number in move_range():  # move right
            alist[sub].append(number)

        for counter, number in enumerate(move_range(), 1):  # move up
            if counter < move_number:  # if it is time to add a new list
                sub -= 1
                alist[sub].append(number)  # add a new element at the rear of existing list
            else:
                alist.insert(0, [number])  # otherwise add the new list to the front
        move_number += 1

        for number in move_range():  # move left
            alist[sub].insert(0, number)

        for counter, number in enumerate(move_range(), 1):  # move down
            sub += 1
            if counter < move_number:  # if it is time to add a new list
                alist[sub].insert(0, number)  # add it to the start of existing list
            else:
                alist.append([number])  # otherwise add a new list to the rear
        move_number += 1
    return alist

In [7]:
import numpy 

def Dany_Rovinsky_spiral(n):
    arr = numpy.array([[1]])
    current, rot_counter = 2, 0
    while current <= n:
        new_row = [numpy.arange(arr.shape[-1]) + current]
        arr = numpy.append(arr, new_row, axis=0)
        arr = numpy.rot90(arr, 3)
        rot_counter += 1
        current = arr[-1, 0] + 1
    rotation_correction = (1 - 3*rot_counter) % 4
    arr = numpy.rot90(arr, rotation_correction)
    return list(filter(lambda x: x <= n, arr.flatten()))

In [8]:
import math

def Damian_Recoskie_spiral( size ):
    XPos = math.sqrt( size ) + 0.5
    YPos = math.ceil( math.sqrt( size ) )
    
    out = [ [ 0 ] * int( XPos ) for _ in range( int( YPos ) ) ]
    
    XPos = int( XPos / 2 + 0.5 )
    YPos = int( YPos / 2 )
    
    Len = 0
    PerLen = 0
    Rotation = 0
    
    for val in range( 1, size + 1 ):
        if Rotation & 1:
            YPos += 1 if Rotation >> 1 else -1
        else:
            XPos += 1 if Rotation >> 1 else -1
        
        out[ YPos ][ XPos ] = val
        
        if Len == PerLen:
            Rotation = ( Rotation + 1 ) & 3
            if Rotation & 1:
                PerLen += 1
            Len = 0
        Len += 1
    return list(reversed(list(zip(*reversed(out)))))


In [9]:
def ray_ziemelis_spiral(number):
    from itertools import cycle
    coordinates = {}

    def coor_gen():
        n = 1
        x, y = 0, 0
        alternator = cycle([1, -1])
        while True:
            alt = next(alternator)
            for _ in range(n):
                yield (x, y)
                x += alt
            for _ in range(n):
                yield (x, y)
                y += alt
            n += 1

    coor = coor_gen()
    for num in range(1, number + 1):
        coordinates[num] = next(coor)

    y_grid_max = max(coordinates[y_value][1] for y_value in coordinates)
    y_grid_min = min(coordinates[y_value][1] for y_value in coordinates)

    coor_slice = {}
    answer, answer_slice = [], []

    for y in range(y_grid_max, y_grid_min - 1, -1):
        for plot in coordinates:
            if coordinates[plot][1] == y:
                coor_slice[coordinates[plot][0]] = plot
        for entry in sorted(coor_slice):
            answer_slice.append(coor_slice[entry])
        answer.append(answer_slice)
        coor_slice = {}
        answer_slice = []

    return answer


# Test Harness

In [10]:
'''
This is a test harness for a numeric spiral builder
Place your function(s) here and run the harness
'''

def test_funcs(*funcs):
    import traceback
    import re
   
    tests = (
        (1,),
        (1, 2),
        (4, 3, 1, 2),
        (5, 4, 3, 6, 1, 2, 7, 8),
        (37, 36, 35, 34, 33, 32, 31,
         38, 17, 16, 15, 14, 13, 30,
         39, 18, 5, 4, 3, 12, 29,
         40, 19, 6, 1, 2, 11, 28,
         41, 20, 7, 8, 9, 10, 27,
         42, 21, 22, 23, 24, 25, 26,
         43, 44, 45, 46, 47, 48, 49)              
    )
    for func in funcs:
        print(f'\n\nTesting function {func.__name__}:\n')
        for expected in tests:
            tested_num = max(expected)
            result = str(func(tested_num))
            result = tuple(filter(None, map(int, re.findall(r'\d+', result))))
            try:
                assert result == expected
            except AssertionError:
                print(f'FAILED for {tested_num}')
                print(f' tested: {result}')
                print(f' expected: {expected}')
            else:
                print(f'Passed for {tested_num}: {result}')
 
 
''' testing functions '''
if __name__ == '__main__':
    func_list = [func for name, func in globals().items()
                 if callable(func) and name[0:2] not in ('__')
                    and name[0:4] not in ('exit', 'quit', 'get_', 'test')]
    test_funcs(*func_list)




Testing function Mark_Geyzer_spiral:

Passed for 1: (1,)
Passed for 2: (1, 2)
Passed for 4: (4, 3, 1, 2)
Passed for 8: (5, 4, 3, 6, 1, 2, 7, 8)
Passed for 49: (37, 36, 35, 34, 33, 32, 31, 38, 17, 16, 15, 14, 13, 30, 39, 18, 5, 4, 3, 12, 29, 40, 19, 6, 1, 2, 11, 28, 41, 20, 7, 8, 9, 10, 27, 42, 21, 22, 23, 24, 25, 26, 43, 44, 45, 46, 47, 48, 49)


Testing function Bartlomiej_Burek_spiral:

Passed for 1: (1,)
Passed for 2: (1, 2)
Passed for 4: (4, 3, 1, 2)
Passed for 8: (5, 4, 3, 6, 1, 2, 7, 8)
Passed for 49: (37, 36, 35, 34, 33, 32, 31, 38, 17, 16, 15, 14, 13, 30, 39, 18, 5, 4, 3, 12, 29, 40, 19, 6, 1, 2, 11, 28, 41, 20, 7, 8, 9, 10, 27, 42, 21, 22, 23, 24, 25, 26, 43, 44, 45, 46, 47, 48, 49)


Testing function Arthur_Zopellaro_spiral:

Passed for 1: (1,)
Passed for 2: (1, 2)
Passed for 4: (4, 3, 1, 2)
Passed for 8: (5, 4, 3, 6, 1, 2, 7, 8)
Passed for 49: (37, 36, 35, 34, 33, 32, 31, 38, 17, 16, 15, 14, 13, 30, 39, 18, 5, 4, 3, 12, 29, 40, 19, 6, 1, 2, 11, 28, 41, 20, 7, 8, 9, 10, 27,